# Imports

In [ ]:
import os
import glob
import json
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import skimage
from skimage.io import imread, imshow
from PIL import Image
import skimage.transform as st	
from tqdm import tqdm
from skimage.transform import resize
from scipy import ndimage


# Creating Dataset and Dataloader

In [ ]:
import os
import tensorflow as tf
import numpy as np

class HubMap_Dataset(tf.keras.utils.Sequence):
    def __init__(self, path, string, img_size, img_arr, mask_arr, batch_size = 4):
        self.path = path
        self.string = string
        self.img_size = img_size
        self.img_arr = img_arr
        self.mask_arr = mask_arr
        self.batch_size = batch_size
    def __len__(self):
        return len(self.img_arr)
    
    def __getitem__(self, index): 
        img_path = self.path + self.string + "images/" + self.img_arr[index]
        mask_path = self.path + self.string + "masks/" + self.mask_arr[index]
        img, mask = Image.open(img_path), Image.open(mask_path)
        img, mask = np.array(img), np.array(mask)
        img = img.astype(np.float32)
        mask = mask.astype(np.float32)
        mask =  np.expand_dims(mask, axis=-1)
        
        return img, mask
    
    
def data_generator(dataset):
    for i in range(len(dataset)):
        yield dataset[i]

In [ ]:
train_img, train_mask = os.listdir("/kaggle/input/hubmapdata/train_images"), os.listdir("/kaggle/input/hubmapdata/train_masks")
valid_img, valid_mask = os.listdir("/kaggle/input/hubmapdata/valid_images"), os.listdir("/kaggle/input/hubmapdata/valid_masks")

In [ ]:
path_to_data = "/kaggle/input/hubmapdata/"
img_size = (512, 512)
batch_size = 4
batch_size1 = 2

train_dataset = HubMap_Dataset(path_to_data, "train_", img_size, train_img, train_mask, batch_size=batch_size)
valid_dataset = HubMap_Dataset(path_to_data, "valid_", img_size, valid_img, valid_mask, batch_size=batch_size1)

## Intialization of the Loader

In [ ]:
data_loader_train = tf.data.Dataset.from_generator(
    generator=lambda: data_generator(train_dataset),
    output_signature=(
        tf.TensorSpec(shape=img_size + (3,), dtype=tf.float32),
        tf.TensorSpec(shape=img_size + (1,), dtype=tf.float32)
    )
)

data_loader_valid = tf.data.Dataset.from_generator(
    generator=lambda: data_generator(valid_dataset),
    output_signature=(
        tf.TensorSpec(shape=img_size + (3,), dtype=tf.float32),
        tf.TensorSpec(shape=img_size + (1,), dtype=tf.float32)
    )
)

In [ ]:
data_loader_train = data_loader_train.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
data_loader_valid = data_loader_valid.batch(batch_size1).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
train_batch = next(iter(data_loader_train))
train_images, train_masks = train_batch

# Image Displays

In [ ]:
train_batch = next(iter(data_loader_train))
valid_batch = next(iter(data_loader_valid))

train_images, train_masks = train_batch
valid_images, valid_masks = valid_batch

# # Plot the first image and mask from the training batch
for i in range(4):
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(train_images[i].numpy().astype(np.uint8))
    plt.title('Training Image')
    plt.subplot(1, 2, 2)
    plt.imshow(train_masks[i].numpy().squeeze(), cmap='gray')
    plt.title('Training Mask')
    plt.show()



In [ ]:
for i in range(2):
    # # Plot the first image and mask from the validation batch
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(valid_images[i].numpy().astype(np.uint8))
    plt.title('Validation Image')
    plt.subplot(1, 2, 2)
    plt.imshow(valid_masks[i].numpy().squeeze(), cmap='gray')
    plt.title('Validation Mask')
    plt.show()

# IoU Loss/Coeff

In [ ]:
def iou_coeff(y_true, y_pred, smooth=1e-7):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_true = tf.squeeze(y_true)
    y_pred = tf.squeeze(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    iou = (intersection + smooth) / (union + smooth)
    return iou


def iou_loss(y_true, y_pred, smooth=1e-7):
    loss = 1.0 - iou_coeff(y_true, y_pred, smooth)
    return loss

# UNet Model + Training

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
# 3x3 Covolutional layer
def double_conv(x, out_channels):
    x = layers.Conv2D(out_channels, kernel_size=3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(out_channels, kernel_size=3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

#Downsampling of the image--> Increasing Features/Decreasing Size
def down(x, out_channels):
    x = layers.MaxPool2D(pool_size=2)(x)
    x = double_conv(x, out_channels)
    return x

#UpSampling of the image--> Decreasing Features/Increasing Size
def up(x1, x2, out_channels, bilinear=True):
    if bilinear:
        x1 = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(x1)
    else:
        x1 = layers.Conv2DTranspose(out_channels // 2, kernel_size=2, strides=2)(x1)

    diffY = x2.shape[1] - x1.shape[1]
    diffX = x2.shape[2] - x1.shape[2]
    x1 = tf.pad(x1, [[0, 0], [diffY // 2, diffY - diffY // 2], [diffX // 2, diffX - diffX // 2], [0, 0]])

    x = tf.concat([x2, x1], axis=-1)
    x = double_conv(x, out_channels)
    return x

#Sigmoid out Activation in order to set numbers to 1 or 0
def out_conv(x, out_channels):
    x = layers.Conv2D(out_channels, kernel_size=1, activation='sigmoid')(x)
    return x

#Creating the model function
def UNet(n_channels=3, n_classes=1, bilinear=True):
    inputs = tf.keras.Input((512,512,3))
    x1 = double_conv(inputs, 64)
    x2 = down(x1, 128)
    x3 = down(x2, 256)
    x4 = down(x3, 512)
    x5 = down(x4, 512)
    x = up(x5, x4, 256, bilinear)
    x = up(x, x3, 128, bilinear)
    x = up(x, x2, 64, bilinear)
    x = up(x, x1, 64, bilinear)
    outputs = out_conv(x, n_classes)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

unet_model = UNet((512, 512, 3))
learning_rate = 0.0001
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
unet_model.compile(optimizer=optimizer, loss=iou_loss, metrics=[iou_coeff])
unet_model.summary()

In [ ]:
#Training the model using .fit()
num_epochs = 150
unet_result = unet_model.fit(data_loader_train, epochs=num_epochs, validation_data = data_loader_valid)

# Saving the model using H5 TensorFlow file type


In [ ]:
# model_json = unet_model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# unet_model.save_weights("model.h5")

# Load The Model

In [ ]:
from keras.models import model_from_json

# load json and create model
json_file = open('/kaggle/input/mode22/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
unet_load = model_from_json(loaded_model_json)
# load weights into new model
unet_load.load_weights("/kaggle/input/mod2thing/model-2.h5")